In [1]:
#This notebook is a first scratch to start an Overwatch League analysis.  
#Written by: Yong-Chan (Liner) Shin

#Load required modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Read data from '20200209_SFS_DAL.csv'
TF = pd.read_csv('20200209_SFS_DAL.csv',engine='python',quotechar='"')
OW_Heroes = pd.read_csv('OW_Heroes.csv')["Heroes"].tolist() #list
T1_players =  pd.read_csv('ROSTERS/SFS_ROSTER.csv')
T2_players =  pd.read_csv('ROSTERS/DAL_ROSTER.csv')
TF.head() #every row is a teamfight
#OW_Heroes.head() #every row is a hero

,MAP,ROUND,TEAM_WON,T1H1,T1P1,T1H2,T1P2,T1H3,T1P3,T1H4,...,T2P2,T2H3,T2P3,T2H4,T2P4,T2H5,T2P5,T2H6,T2P6,ACT_LOG
0,OASIS,1,SFS,MCCREE,STRIKER,MEI,SINATRAA,REINHARDT,SUPER,DVA,...,DECAY,ORISA,GAMSU,REINHARDT,TRILL,BAPTISTE,CRIMZO,LUCIO,CLOSER,"DECAY(CLOSER)->SUPER,CHOIHYOBIN(MOTH/STRIKER)-..."
1,OASIS,1,SFS,MCCREE,STRIKER,MEI,SINATRAA,REINHARDT,SUPER,DVA,...,DECAY,ORISA,GAMSU,REINHARDT,TRILL,BAPTISTE,CRIMZO,LUCIO,CLOSER,"STRIKER->ULT,STRIKER->TRILL,STRIKER->GAMSU,DEC..."
2,OASIS,1,DAL,TRACER,STRIKER,MEI,SINATRAA,REINHARDT,SUPER,DVA,...,DECAY,ORISA,GAMSU,REINHARDT,TRILL,BAPTISTE,CRIMZO,LUCIO,CLOSER,"SINATRAA->ULT,MOTH->ULT->(STRIKER/SINATRAA/SUP..."
3,OASIS,1,SFS,TRACER,STRIKER,MEI,SINATRAA,REINHARDT,SUPER,DVA,...,DECAY,ORISA,GAMSU,REINHARDT,TRILL,BAPTISTE,CRIMZO,LUCIO,CLOSER,"ARCHITECT->ULT,STRIKER->ULT,CLOSER(GAMSU)->SIN..."
4,OASIS,1,SFS,TRACER,STRIKER,DOOMFIST,SINATRAA,REINHARDT,SUPER,DVA,...,DECAY,ORISA,GAMSU,REINHARDT,TRILL,MOIRA,CRIMZO,LUCIO,CLOSER,"TRILL->ULT,DECAY(GAMSU/TRILL)->ARCHITECT,SINAT..."


In [2]:
T1_hero_used_df=TF.iloc[:,[2*i+1 for i in range(1,7)]]
T2_hero_used_df=TF.iloc[:,[2*i+1 for i in range(7,13)]]
T1_player_played_df=TF.iloc[:,[2*i+2 for i in range(1,7)]]
T2_player_played_df=TF.iloc[:,[2*i+2 for i in range(7,13)]]
#T1_player_played_df.head()
T1_hero_used_df.head()
#T1_hero_used_list=[for key, value in T1_hero_used_df.iteritems()]

,T1H1,T1H2,T1H3,T1H4,T1H5,T1H6
0,MCCREE,MEI,REINHARDT,DVA,ANA,LUCIO
1,MCCREE,MEI,REINHARDT,DVA,ANA,LUCIO
2,TRACER,MEI,REINHARDT,DVA,ANA,LUCIO
3,TRACER,MEI,REINHARDT,DVA,ANA,LUCIO
4,TRACER,DOOMFIST,REINHARDT,DVA,ANA,LUCIO


In [3]:
#Take in a TF log and return each player's Level of Contribution
#'()->': LEFT final blows RIGHT with helps in the PARENTHESIS or LEFT changes to RIGHT
#'->ULT->':LEFT used ult for RIGHT(Only for Lucio now)
#'->ULT<-':LEFT used ult but canceled by RIGHT(e.g. Ana, Dva, etc.)
def Interpreter(an_action):
    #Interpreter returns following cases: 
    #1. [hero_changed,player,hero_changed]
    #2. [hero_not_changed,player,ult,canceler or ULT_efficiency] canceler or ULT_efficiency may not exist
    #3. [hero_not_changed,final_blower+kill_helpers,dead] helpers may not exist
    hero_change_flag=False 
    #print("current action:",an_action)
    LEFT,RIGHT = an_action.split('->',1)
    #print("current LEFT:",LEFT)
    #print("current RIGHT:",RIGHT)
    if RIGHT in OW_Heroes: #check if an action is just hero change
        #print("Enter change hero case")
        hero_change_flag=True
        return [hero_change_flag,LEFT,RIGHT]
    if 'ULT' in RIGHT: #check if an action is just ULT
        if '<-' in RIGHT: #Enemy canceled an ULT with a common skill
            RIGHT_canceler = RIGHT.split('<-')[1]
            return [hero_change_flag,LEFT,'ULT',RIGHT_canceler]
        elif '->' in RIGHT: #check if the ULT is Lucio's Soundbarrier
            ULT_efficiency = len(RIGHT.split('->')[1].split('/'))/5.0
            return [hero_change_flag,LEFT,'ULT',ULT_efficiency]
        else:
            return [hero_change_flag,LEFT,'ULT']
    #Final blow happened
    if '(' in LEFT: #Final blow happens with teammates' help
        LEFT_final_blow,LEFT_helps = LEFT.split('(')
        #print("current final_blow:",LEFT_final_blow)
        if '/' in LEFT:
            LEFT_helps = LEFT_helps.split('/')
            LEFT_helps[len(LEFT_helps)-1]=LEFT_helps[len(LEFT_helps)-1].rsplit(')',1)[0]
        else:
            LEFT_helps = LEFT_helps.rsplit(')',1)
            if LEFT_helps[1]=='':
                LEFT_helps = [LEFT_helps[0]]
        #print("current LEFT_helps:",LEFT_helps)
        LEFT = [LEFT_final_blow,LEFT_helps]
    else: #Solo kill (according to kill log)
        LEFT = [LEFT]
    return [hero_change_flag,LEFT,RIGHT]

    
def LoC_TF(TF,TF_idx,score_dict):
    
    ult_score = 1.0 #ult score can approximate damage dealed/healed
    ult_efficiency = 1.0 #maybe decreased for soundbarrier
    kill_score = 2.0 #the score gain if places final blow
    death_penalty = -2.0 # the score lost if dead
    k=1 #logistic growth rate
    x0=3 #mid-point of logistic func, fixed as x is between 1-5 always
    final_blower_portion_func = lambda num_participated: 1.0-kill_score/(1.0+np.exp(-1.0*k*(num_participated-x0)))
    #kill_LB = final_blower_portion_func(5)
    action_list = TF[TF_idx].split(',')
    for action in action_list:
        action_interpreted = Interpreter(action)
        #print("current action_interpreted:",action_interpreted)
        #Interpreter returns following cases: 
        #1. [hero_changed,player,hero_changed]
        #2. [hero_not_changed,player,'ULT',canceler or ULT_efficiency] canceler or ULT_efficiency may not exist
        #3. [hero_not_changed,[final_blower+kill_helpers],dead] helpers may not exist
        if action_interpreted[0]==False: #consider cases that hero is not changed for now
            if action_interpreted[2]=='ULT': #if the action is ULT
                player_ult = action_interpreted[1]
                if len(action_interpreted)==4: #if there is a canceler or the ULT is soundbarrier
                    if type(action_interpreted[3]) is str:
                        ult_canceler = action_interpreted[3]
                        score_dict[ult_canceler]+=ult_score
                    else: # if type(action_interpreted[3]) is float
                        ult_efficiency = action_interpreted[3]
                score_dict[player_ult]+=ult_score*ult_efficiency
                ult_efficiency=1.0
            else: #if the action is final blow
                players_in_kill = action_interpreted[1]
                #print("players_in_kill",players_in_kill)
                final_blower = players_in_kill[0]
                if len(players_in_kill)==1: #if it is a solo kill
                    score_dict[final_blower]+=kill_score
                else: #if it is not a solo kill
                    kill_helpers = players_in_kill[1]
                    final_blower_portion = final_blower_portion_func(len(kill_helpers))
                    score_dict[final_blower]+=kill_score*final_blower_portion
                    for helper in kill_helpers:
                        score_dict[helper]+=kill_score*(1-final_blower_portion)/len(kill_helpers)
                dead_player = action_interpreted[2]
                score_dict[dead_player]+=death_penalty

score_dict={}
for T1_player in T1_player_played_df.iloc[0]: #assume no player change
    score_dict[T1_player]=[] #initialize T1 score with empty list
for T2_player in T2_player_played_df.iloc[0]: #assume no player change
    score_dict[T2_player]=[] #initialize T2 score with empty list
    
for idx in range(0,TF['ACT_LOG'].size): #iterate through every TF
    curr_score_dict={}
    for T1_player in T1_player_played_df.iloc[0]: #assume no player change
        curr_score_dict[T1_player]=0 #initialize T1 score with 0
    for T2_player in T2_player_played_df.iloc[0]: #assume no player change
        curr_score_dict[T2_player]=0 #initialize T2 score with 0
    #print("curr idx:",idx)
    LoC_TF(TF['ACT_LOG'],idx,curr_score_dict)
    for player in score_dict:
        score_dict[player].append(curr_score_dict[player])
score_df = pd.DataFrame(score_dict)
score_df.head()

,STRIKER,SINATRAA,SUPER,CHOIHYOBIN,ARCHITECT,MOTH,DOHA,DECAY,GAMSU,TRILL,CRIMZO,CLOSER
0,0.537883,3.523188,-2.000000,3.970611,-1.523188,1.491506,-2.000000,-0.476812,-1.046377,-0.476812,-2.000000,0.000000
1,3.000000,-2.000000,0.000000,0.000000,1.000000,0.000000,0.000000,4.000000,-2.000000,-2.000000,0.000000,0.000000
2,-2.000000,-0.523188,-3.000000,-3.000000,0.000000,2.523188,4.523188,6.046377,2.907247,-2.000000,2.523188,0.800000
3,1.924234,2.985305,-0.476812,0.924234,4.567272,1.075766,-1.462117,-1.000000,-0.598954,-2.000000,-1.000000,0.061071
4,6.046377,2.000000,1.476812,1.000000,-1.523188,2.800000,-1.000000,0.924234,-1.462117,-0.462117,-2.000000,-2.000000


In [4]:
score_df['TEAM_WON'] = TF['TEAM_WON']
score_df.head()

,STRIKER,SINATRAA,SUPER,CHOIHYOBIN,ARCHITECT,MOTH,DOHA,DECAY,GAMSU,TRILL,CRIMZO,CLOSER,TEAM_WON
0,0.537883,3.523188,-2.000000,3.970611,-1.523188,1.491506,-2.000000,-0.476812,-1.046377,-0.476812,-2.000000,0.000000,SFS
1,3.000000,-2.000000,0.000000,0.000000,1.000000,0.000000,0.000000,4.000000,-2.000000,-2.000000,0.000000,0.000000,SFS
2,-2.000000,-0.523188,-3.000000,-3.000000,0.000000,2.523188,4.523188,6.046377,2.907247,-2.000000,2.523188,0.800000,DAL
3,1.924234,2.985305,-0.476812,0.924234,4.567272,1.075766,-1.462117,-1.000000,-0.598954,-2.000000,-1.000000,0.061071,SFS
4,6.046377,2.000000,1.476812,1.000000,-1.523188,2.800000,-1.000000,0.924234,-1.462117,-0.462117,-2.000000,-2.000000,SFS


In [5]:
score_df.sum(axis=0,numeric_only=True)

STRIKER       26.943515
SINATRAA      16.141423
SUPER          1.974315
CHOIHYOBIN    28.360667
ARCHITECT     20.539701
MOTH           5.240379
DOHA         -17.432728
DECAY         23.398991
GAMSU          9.350083
TRILL         -5.116145
CRIMZO        -0.581966
CLOSER         8.981765
dtype: float64

First of all, I want to confirm that the values here depend on multiple variable and the reasons for every statistical values depend on the teammates as well, so the high/low values absolutely do not imply that the player is doing good or bad. What we want to figure out is the contribution of teamfight for every player/hero and discuss what tactics should be added/modified for victory in the teamfight.

For convenience, let TF considered as Teamfight and ULT as ultimate skill. So the result above gives us a rough insight about the standings of every player in overall TF's. However, we have to count that STRIKER and DECAY have larger contributions than SINATRAA and DOHA becasue STRIKER and DECAY mostly played high damage dps like REAPER while SINATRAA and DOHA mostly played MEI, and the data is not counting any of MEI's wall while it is powerful enough to have a huge impact on the result of a TF(especially at the maps with narrow roads like King's Row). The problem is that the kill log only reflects MEI's primary/secondary fires and ULT.

In [6]:
SINATRAA_vs_DOHA=pd.DataFrame(index=OW_Heroes)
SINATRAA_vs_DOHA['SINATRAA']=TF.groupby(['T1H2']).agg("count")['T1P2']
SINATRAA_vs_DOHA['DOHA'] = TF.groupby(['T2H1']).agg("count")['T2P1']
SINATRAA_vs_DOHA.dropna(how='all')

,SINATRAA,DOHA
DOOMFIST,14.0,NaN
MEI,16.0,37.0
SOMBRA,NaN,1.0
SYMMETRA,1.0,NaN
TRACER,7.0,NaN


Both SINATRAA and DOHA focused on MEI, but SINATRAA used more various heroes while DOHA highly focused on MEI. So this was the why SINATRAA obtains much higher score. 

In [7]:
T1_vs_T2_Tanks=pd.DataFrame(index=OW_Heroes)
T1_vs_T2_Tanks['SUPER']=TF.groupby(['T1H3']).agg("count")['T1P3']
T1_vs_T2_Tanks['CHOIHYOBIN'] = TF.groupby(['T1H4']).agg("count")['T1P4']
T1_vs_T2_Tanks['GAMSU']=TF.groupby(['T2H3']).agg("count")['T2P3']
T1_vs_T2_Tanks['TRILL'] = TF.groupby(['T2H4']).agg("count")['T2P4']
T1_vs_T2_Tanks.dropna(how='all')

,SUPER,CHOIHYOBIN,GAMSU,TRILL
DVA,NaN,32.0,NaN,NaN
ORISA,NaN,NaN,38.0,NaN
REINHARDT,38.0,NaN,NaN,38.0
SIGMA,NaN,6.0,NaN,NaN


We can see a hugh difference between the two teams' tanks as well. Among both teams, CHOIHYOBIN's score is definitely remarkable. The difference between the two team was the existence of Off-tank(in this case, let's focus on DVA more than SIGMA). Since Off-tank has high mobility, high instantaneous damage with  defensive matrix to negate multiple ults, DVA is conative tank that has high potential to attain high score. On the other hand, REINHARDT and ORISA has low mobility and requires passive play-style which results in low scores. Thus, SFS depends less on DPS while DAL highly depends on thee final blow skills of DPS. Furthermore, DOHA is binded to play MEI, so DAL solely depends on Decay's final blows to win any TF. 

In [8]:
T1_vs_T2_Tanks=pd.DataFrame(index=OW_Heroes)
T1_vs_T2_Tanks['ARCHITECT']=TF.groupby(['T1H5']).agg("count")['T1P5']
T1_vs_T2_Tanks['MOTH'] = TF.groupby(['T1H6']).agg("count")['T1P6']
T1_vs_T2_Tanks['CRIMZO']=TF.groupby(['T2H5']).agg("count")['T2P5']
T1_vs_T2_Tanks['CLOSER'] = TF.groupby(['T2H6']).agg("count")['T2P6']
T1_vs_T2_Tanks.dropna(how='all')

,ARCHITECT,MOTH,CRIMZO,CLOSER
ANA,38.0,NaN,NaN,NaN
BAPTISTE,NaN,NaN,19.0,NaN
BRIGITTE,NaN,5.0,NaN,NaN
LUCIO,NaN,33.0,NaN,38.0
MOIRA,NaN,NaN,19.0,NaN


Lastly, let's talk about the healers. SFS fixed their healers' comp as ANA and LUCIO, and sometime ANA and BRIGITTE when they are in last point defense on map Eichenwalde. On the other hand, DAL fix Closer as LUCIO, CRIMZO plays both BAPTISTE and MOIRA. Strange thing is that CRIMZO's hero pick seems quite arbitrary. In most case, when a round starts, CRIMZO chooses BAPTISTE, but he eventually changes to MOIRA, so we can think that DAL did not really fix CRIMZO's pick or CRIMZO was not sure about his BAPTISTE play. Additionally, ARCHITECT's ANA had a hugh impact on overall TF's. We need more data to figure out if this is the hero's characteristics over BAPTISTE and MOIRA, or it's ARCHITECT's techniques, but it is confident that ARCHITECT's ANA has a phenomenal impact on TF's and according to the score board above, his impact is larger than SINATRAA's. This may be why ARCHITECT was the MVP of the match.

This is a very, very rough draft to start an analysis on OWL match. It took about 9 hours starting from nothing, and most of the time was spent on manually collecting data from the OWL Viewer, so if I can find any way to shorten the data collecting time, I can focus more on modifying parameters and inserting complicated values for deeper analysis. Lastly, I look forward to use ML algorithms to eventually give a direction to achieve higher win rate in TF's, ultimately to achieve victory in any match. 